In [1]:
import lmoments3 as lm
from lmoments3 import distr
import pandas as pd
import numpy as np
import os
from scipy.stats import kappa4
from scipy.special import gamma
from scipy.optimize import minimize

In [2]:
results_folder = r'../example/results/heterogeneity'
if not os.path.exists(results_folder):
    os.mkdir(results_folder)

In [13]:
stations = ['Station1','Station2','Station3','Station4','Station5','Station6','Station7'] #Selection of stations of interest within a cluster
df_prec = pd.read_parquet('precipitations_daily_time_series.parquet')
lmoments_list_val = list()
for sta, df_day in df_prec.groupby('StationId'): 
    #Extracting data from db, clearing NANs and passing it into an array
    if(sta not in stations):continue #Error values in station 8
    df_station = df_day[df_day["StationId"]==sta]
    station = df_station[df_station['IR']>=2]['IR']
    array = station.to_numpy()
    x = array[~np.isnan(array)]
    n_x = len(x)
    sum_x = sum(x)
    #L-moment determination
    lamb1, lamb2, t3, t4 = lm.lmom_ratios(x, nmom=4)
    l_CV = lamb2/lamb1
    lmoments_list_val.append([sta, lamb1, lamb2, t3, t4, l_CV, n_x, sum_x])

df_lmoments = pd.DataFrame(lmoments_list_val, columns =['station', 'lamb1', 'lamb2', 't3', 't4', 'l_CV', 'n', 'sum_x'])
df_lmoments.to_excel(os.path.join(results_folder, 'lmoments.xlsx'), index=False)

In [14]:
#FITTING KAPPA 4 DISTRIBUTION

#Determining V from observations
N = len(df_lmoments)
tR_den = 0
Num = 0
V_den = 0
lamb1_mean_den = 0
lamb2_mean_den = 0
l_CV_mean_den = 0
tau3_mean_den = 0
tau4_mean_den = 0

for i in range(0, N):
    tR_den = tR_den + df_lmoments.at[i, 'n']*df_lmoments.at[i, 'l_CV']
    lamb1_mean_den = lamb1_mean_den + df_lmoments.at[i, 'n']*df_lmoments.at[i, 'lamb1']
    lamb2_mean_den = lamb2_mean_den + df_lmoments.at[i, 'n']*df_lmoments.at[i, 'lamb2']
    l_CV_mean_den = l_CV_mean_den + df_lmoments.at[i, 'n']*df_lmoments.at[i, 'l_CV']
    tau3_mean_den = tau3_mean_den + df_lmoments.at[i, 'n']*df_lmoments.at[i, 't3']
    tau4_mean_den = tau4_mean_den + df_lmoments.at[i, 'n']*df_lmoments.at[i, 't4']
    Num = Num + df_lmoments.at[i, 'n']
        
Mean_tR = tR_den/Num
t3_reg = tau3_mean_den/Num
t4_reg = tau4_mean_den/Num
lamb1_reg = lamb1_mean_den/Num
lamb2_reg = lamb2_mean_den/Num
l_CV_reg = l_CV_mean_den/Num
    
#print('Weighted means: t =', l_CV_reg, ', t_3 =',t3_reg, ', t_4 =', t4_reg)

for i in range(0,N):
    V_den = V_den + df_lmoments.at[i, 'n']*(df_lmoments.at[i, 'l_CV']-Mean_tR)**2

V = (V_den/Num)**0.5

In [ ]:
#Fitting Kappa4
#g_r functions
def calculate_g_r(h, k, r):
    if h > 0:
        return (r * gamma(1 + k) * gamma(r / h)) / (h**(1 + k) * gamma(1 + k + (r / h)))
    else:
        return (r * gamma(1 + k) * gamma(-k - (r / h))) / ((-h)**(1 + k) * gamma(1 - (r / h)))


def objective_function(params):
    h, k = params 

    #g_r functions calculations
    g1 = calculate_g_r(h, k, 1)
    g2 = calculate_g_r(h, k, 2)
    g3 = calculate_g_r(h, k, 3)
    g4 = calculate_g_r(h, k, 4)

    #t3' y t4' calculations
    t3_prime = (-g1 + 3 * g2 - 2 * g3) / (g1 - g2)
    t4_prime = (-g1 + 6 * g2 - 10 * g3 + 5 * g4) / (g1 - g2)

    #error calculation
    diff_t3 = (np.abs(t3_prime) - np.abs(t3_reg))**2
    diff_t4 = (np.abs(t4_prime) - np.abs(t4_reg))**2
    error = (np.abs(diff_t3) + np.abs(diff_t4))**0.5

    return error


x0 = np.array([0.1, 0])
    
res = minimize(objective_function, x0, method='nelder-mead', options={'xatol': 1e-8, 'disp': False})
x_min = res.x

#k and h saving values
best_h = x_min[0]
best_k = x_min[1]
    
#alpha and beta calculations
    
alpha = (l_CV_reg * best_k) / (calculate_g_r(best_h, best_k, 1) - calculate_g_r(best_h, best_k, 2))
locat = 1-(1-calculate_g_r(best_h, best_k, 1)) * alpha / best_k
    
print('Parameters of regional kappa distribution: loc =', locat, ', alpha =', alpha, ', k =', best_k, ', h =', best_h)

J = (best_h, best_k, locat, alpha)

In [ ]:
#Simulating Monte Carlo series

N_sites = N
N_sim = 1000
lmoments_sim_val = list()
V_sim = list()
for d1 in range(N_sim):
    TR_den = 0
    num = 0
    L_cv_val = list()
    for d2 in range(N_sites):
        random_samples = kappa4.rvs(J[0], J[1], loc=J[2], scale=J[3], size=df_lmoments.at[d2, 'n'])
        #L-moment determination
        l1, l2, tau3, tau4 = lm.lmom_ratios(random_samples, nmom=4)
        L_cv = l2/l1
        n_prima = len(random_samples)
        lmoments_sim_val.append([d1, d2, l1, l2, tau3, tau4, L_cv, n_prima])
        L_cv_val.append([d2, L_cv, n_prima])
        TR_den = TR_den + df_lmoments.at[d2, 'n']*L_cv
        num = num + df_lmoments.at[d2, 'n']
        df_L_cv = pd.DataFrame(L_cv_val, columns =['Sites','L_cv', 'n_prima'])
        
    Mean_TR = TR_den/num
    V1_den = 0
    for d3 in range(N_sites):
        V1_den = V1_den + df_L_cv.at[d3, 'n_prima']*(df_L_cv.at[d3, 'L_cv']- Mean_TR)**2
        
    V1 = (V1_den/(num))**0.5
    V_sim.append([d1, V1])
        
#df_lmoments_sim_val = pd.DataFrame(lmoments_sim_val, columns =['Sim', 'Sites', 'l1', 'l2', 'tau3', 'tau4', 'L_cv', 'n_prima'])
#df_lmoments_sim_val.to_excel(os.path.join(results_folder, 'lmoments_sim.xlsx'), index=False)
df_V1 = pd.DataFrame(V_sim, columns =['Sim', 'V'])
#df_V1.to_excel(os.path.join(results_folder, 'V_sim.xlsx'), index=False)

#Calculating the Heterogeneity

df_V = df_V1['V']
vals = df_V.to_numpy()
V_sim_mean = np.mean(vals)
V_sim_std = np.std(vals)

H = (V-V_sim_mean)/V_sim_std

print(H)